In [1]:
import numpy as np
import cvxpy as cp

## DATASET

In [2]:
C = np.matrix([[8, 6, 10, 9, 8], [9, 12, 13, 7, 5], [14, 9, 16, 5, 2]])
S = np.array([40, 50, 45]).reshape(-1, 1)
D = np.array([45, 20, 30, 30, 10]).reshape(-1, 1)

## CVX problem formulation
1. **C** is cost matrix.
2. $\overline{S}$ is total supply.
3. $\overline{D}$ is total demand.
4. **A** is the startegy matrix.

Total cost = $\mathbf{C}^T\mathbf{A}$
**Constraints**
1. Positive supplies, $\mathbf{A} \ge 0$
2. Meet the demand, $\mathbf{A}^T\overline{\mathbf{1}} = \overline{D}$
3. Restriction on maximum supply, $\mathbf{A}\overline{\mathbf{1}} \le \overline{S}$

In [3]:
A = cp.Variable((C.shape[0], C.shape[1]))

ones_1 = np.ones((C.shape[0], 1))
ones_2 = np.ones((C.shape[1], 1))

Constraints = [A.T@ones_1 == D, A@ones_2 <= S, A >= 0]
cost = cp.trace(C.T@A)

In [4]:
Objective = cp.Minimize(cost)

problem = cp.Problem(objective=Objective, constraints= Constraints)
problem.solve();

In [5]:
# Print optimal transportation
A_value = np.round(A.value, 2)
from prettytable import PrettyTable

table = PrettyTable(["Factory", "D1", "D2", "D3", "D4", "D5", "Supply"])
for i in range(C.shape[0]):
    table.add_row(["Factory {}".format(i+1), A_value[i, 0], A_value[i, 1], A_value[i, 2], 
                   A_value[i, 3], A_value[i, 4], np.sum(A_value[i, :])])
print(table)

demand_table = PrettyTable(["Demand", "D1", "D2", "D3", "D4", "D5"])
demand_table.add_row(["", np.sum(A_value[:, 0]), np.sum(A_value[:, 1]), np.sum(A_value[:, 2]), 
                            np.sum(A_value[:, 3]), np.sum(A_value[:, 4])])
print(demand_table)
print("################ Total Cost #################")
print("################   {}   #################".format(np.round(cost.value, 3)))

+-----------+------+------+------+------+------+--------+
|  Factory  |  D1  |  D2  |  D3  |  D4  |  D5  | Supply |
+-----------+------+------+------+------+------+--------+
| Factory 1 | 0.0  | 15.0 | 25.0 | 0.0  | 0.0  |  40.0  |
| Factory 2 | 45.0 | 0.0  | 5.0  | 0.0  | 0.0  |  50.0  |
| Factory 3 | 0.0  | 5.0  | 0.0  | 30.0 | 10.0 |  45.0  |
+-----------+------+------+------+------+------+--------+
+--------+------+------+------+------+------+
| Demand |  D1  |  D2  |  D3  |  D4  |  D5  |
+--------+------+------+------+------+------+
|        | 45.0 | 20.0 | 30.0 | 30.0 | 10.0 |
+--------+------+------+------+------+------+
################ Total Cost #################
################   1025.0   #################
